<a href="https://colab.research.google.com/github/iceivy/Data-Analysis/blob/master/SD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 装载谷歌云端硬盘到虚拟机

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 克隆github项目到虚拟机sample_data文件夹

In [ ]:
!git clone https://github.com/davidstap/AttnGAN.git

* 解压缩文件
* 返回上一级：cd ..
* 查看当前目录：pwd
* 查看当前目录里的所有文件和文件夹：ls
* 解压zip格式的压缩文件（text.zip为目标文件）：unzip text.zip
* 解压tar格式的压缩文件（CUB_200_2011.tgz为目标文件）：!tar -xf CUB_200_2011.tgz

In [ ]:
#!cd sample_data/AttnGAN/data/
!unzip sample_data/AttnGAN/data/birds.zip

In [ ]:
!unzip sample_data/AttnGAN/data/birds/text.zip

In [ ]:
! python pretrain_DAMSM.py --cfg cfg/DAMSM/bird.yml --gpu 0
! python main.py --cfg cfg/bird_attn2.yml --gpu 0
! python main.py --cfg cfg/eval_bird.yml --gpu 0

* 执行命令行前面加！
* 自带库及库的安装Colab 自带了Pytorch， Tensorflow、Matplotlib、Numpy、Pandas 等深度学习基础库。如果还需要其他依赖，如 Keras，可以新建代码块，利用pip安装
* 连续使用超过12小时会被限制，使用时中途保存模型文件
* 如果虚拟机长时间未检测到你有什么指令和动作，会自动切断连接，一切就得重新来。

# SexismDetection

In [3]:
cd /content/sample_data/SexismDetection/

/content/sample_data/SexismDetection


In [10]:
# 环境准备
!git lfs install
!git clone https://huggingface.co/bert-base-chinese
# GIT_LFS_SKIP_SMUDGE=1
!pip install transformers

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'bert-base-chinese'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49
Unpacking objects: 100% (49/49), 156.82 KiB | 669.00 KiB/s, done.
Filtering content: 100% (4/4), 1.59 GiB | 80.23 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.6 MB/s eta 0:00:00


In [4]:
#%%导入需要的库
import numpy as np
import pandas as pd
import csv

import torch
import torch.nn as nn
import torch.functional as F 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset, DataLoader

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tqdm import tqdm

#from transformers import BertTokenizer,BertConfig,AdamW

In [6]:
class MyDataSet(Dataset):
    def __init__(self, loaded_data):
        self.data = loaded_data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
 
Data_path = "data/JD_Bert.csv"
Totle_data = pd.read_csv(Data_path)
data=[(Totle_data['contents'][i],Totle_data['labels'][i]) for i in range(0,len(Totle_data))]#为语料加上标签
import random
random.seed(1)#若想重现，保留此句
random.shuffle(data)
random.shuffle(data)
rate=0.8#训练集的比例
train_dataset=data[:int(len(data)*rate)]#训练集
validate_dataset=data[int(len(data)*rate):int(len(data)*(rate+0.1))]#测试集
test_dataset=data[int(len(data)*(rate+0.1)):]#测试集

In [7]:
print(len(data),len(train_dataset),len(validate_dataset),len(test_dataset))

8962 7169 896 897


In [8]:
#设置保存路径
train_data_path="data/JD_Bert_Try.csv"
dev_data_path = "data/JD_Bert_Dev.csv" 
test_data_path="data/JD_Bert_Test.csv"
 
#index参数设置为False表示不保存行索引,header设置为False表示不保存列索引
pd.DataFrame(train_dataset).to_csv(train_data_path,index=False,header=['contents', 'labels'])
pd.DataFrame(validate_dataset).to_csv(dev_data_path,index=False,header=['contents', 'labels'])
pd.DataFrame(test_dataset).to_csv(test_data_path,index=False,header=['contents', 'labels'])
#%%查看数据
data0 = pd.read_csv(train_data_path)
data0.head(2)

,contents,labels
0,他是狗娘养的自然就觉得女的不配当人呗,1
1,对她来说因为没有证据可能永远都不能让法律制裁伍迪所以她只能在这种时候想用舆论压死他可是这件事...,0


In [11]:
#%%定义神经网络
import transformers
from transformers import BertTokenizer,BertConfig,AdamW
class BertClassificationModel(nn.Module):
    def __init__(self):
        super(BertClassificationModel, self).__init__()   
        #加载预训练模型
        pretrained_weights="bert-base-chinese/"
        self.bert = transformers.BertModel.from_pretrained(pretrained_weights)
        for param in self.bert.parameters():
            param.requires_grad = True
        #定义线性函数      
        self.dense = nn.Linear(768, 2)  #bert默认的隐藏单元数是768， 输出单元是2，表示二分类
        
    def forward(self, input_ids,token_type_ids,attention_mask):
        #得到bert_output
        bert_output = self.bert(input_ids=input_ids,token_type_ids=token_type_ids, attention_mask=attention_mask)
        #获得预训练模型的输出
        bert_cls_hidden_state = bert_output[1]
        #将768维的向量输入到线性层映射为二维向量
        linear_output = self.dense(bert_cls_hidden_state)
        return  linear_output

#%%使用BertTokenizer 编码成Bert需要的输入格式
'''
数据送入预训练模型之前需要进行预处理，使用BertTokenizer将数据编码为Bert需要的输入格式。
预训练模型有三种输入分别是input_ids、token_type_ids 、attention_mask。
参考： https://zhuanlan.zhihu.com/p/160813500
'''
def encoder(max_len,vocab_path,text_list):
    #将text_list embedding成bert模型可用的输入形式
    #加载分词模型
    tokenizer = BertTokenizer.from_pretrained(vocab_path)
    tokenizer = tokenizer(
        text_list,
        padding = True,
        truncation = True,
        max_length = max_len,
        return_tensors='pt'  # 返回的类型为pytorch tensor
        )
    input_ids = tokenizer['input_ids']
    token_type_ids = tokenizer['token_type_ids']
    attention_mask = tokenizer['attention_mask']
    return input_ids,token_type_ids,attention_mask
#%%将数据加载为Tensor格式
def load_data(path):
    d=pd.read_csv(path)
    text_list=[d['contents'][i] for i in range(0,len(d))]
    labels=[int(d['labels'][i]) for i in range(0,len(d))]
    #调用encoder函数，获得预训练模型的三种输入形式
    input_ids,token_type_ids,attention_mask = encoder(max_len=150,vocab_path="bert-base-chinese/vocab.txt",text_list=text_list)
    labels = torch.tensor(labels)
    #将encoder的返回值以及label封装为Tensor的形式
    data = TensorDataset(input_ids,token_type_ids,attention_mask,labels)
    return data

#%%实例化DataLoader
#设定batch_size
batch_size = 16
#调用load_data函数，将数据加载为Tensor形式
#设置保存路径
train_data_path="data/JD_Bert_Try.csv"
dev_data_path = "data/JD_Bert_Dev.csv" 
test_data_path="data/JD_Bert_Test.csv"
train_data = load_data(train_data_path)
dev_data = load_data(dev_data_path)
test_data = load_data(test_data_path)
#将训练数据和测试数据进行DataLoader实例化
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dataset=dev_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)  

#%%定义验证函数
def dev(model,dev_loader):
    #将模型放到服务器上
    model.to(device)
    #设定模式为验证模式
    model.eval()
    #设定不会有梯度的改变仅作验证
    with torch.no_grad():
        correct = 0
        total = 0
        for step, (input_ids,token_type_ids,attention_mask,labels) in tqdm(enumerate(dev_loader),desc='Dev Itreation:'):
            input_ids,token_type_ids,attention_mask,labels=input_ids.to(device),token_type_ids.to(device),attention_mask.to(device),labels.to(device)
            out_put = model(input_ids,token_type_ids,attention_mask)
            _, predict = torch.max(out_put.data, 1)
            correct += (predict==labels).sum().item()
            total += labels.size(0)
        res = correct / total
        return res

#%%定义训练函数
def train(model,train_loader,dev_loader) :
    #将model放到服务器上
    model.to(device)
    #设定模型的模式为训练模式
    model.train()
    #定义模型的损失函数
    criterion = nn.CrossEntropyLoss()
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    #设置模型参数的权重衰减
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    #学习率的设置
    optimizer_params = {'lr': 1e-5, 'eps': 1e-6, 'correct_bias': False}
    #使用AdamW 主流优化器
    optimizer = AdamW(optimizer_grouped_parameters, **optimizer_params)
    #学习率调整器，检测准确率的状态，然后衰减学习率
    scheduler = ReduceLROnPlateau(optimizer,mode='max',factor=0.5,min_lr=1e-7, patience=5,verbose= True, threshold=0.0001, eps=1e-08)
    t_total = len(train_loader)
    #设定训练轮次
    total_epochs = 2
    bestAcc = 0
    correct = 0
    total = 0
    print('开始训练!!!')
    for epoch in range(total_epochs): 
        from tqdm.auto import tqdm
        total_loss=0
        step=0
        for batch in tqdm(train_loader,desc=f'{epoch}'):
            step=step+1
            #梯度清零
            optimizer.zero_grad()

            #从实例化的DataLoader中取出数据，并通过 .to(device)将数据部署到服务器上
            #将数据输入到模型中获得输出
            input_ids,token_type_ids,attention_mask,labels=[x.to(device) for x in batch]
            out_put =  model(input_ids,token_type_ids,attention_mask)
                   
            #计算损失
            loss = criterion(out_put, labels)
            _, predict = torch.max(out_put.data, 1)
            correct += (predict == labels).sum().item()
            total += labels.size(0)
            loss.backward()
            optimizer.step()
            #每两步进行一次打印
            if (step + 1) % 50 == 0:
                train_acc = correct / total
                print("Train Epoch[{}/{}],step[{}/{}],tra_acc{:.6f} %,loss:{:.6f}".format(epoch + 1, total_epochs, step + 1, len(train_loader),train_acc*100,loss.item()))
            #每五十次进行一次验证
            if (step + 1) % 100 == 0:
                train_acc = correct / total
                #调用验证函数dev对模型进行验证，并将有效果提升的模型进行保存
                acc = dev(model, dev_loader)
                if bestAcc < acc:
                    bestAcc = acc
                    #模型保存路径
                    path = 'data/span_bert_hide_model1.pkl'
                    torch.save(model, path)
                print("DEV Epoch[{}/{}],step[{}/{}],tra_acc{:.6f} %,bestAcc{:.6f}%,dev_acc{:.6f} %,loss:{:.6f}".format(epoch + 1, total_epochs, step + 1, len(train_loader),train_acc*100,bestAcc*100,acc*100,loss.item()))
        scheduler.step(bestAcc)
#%%实例化模型并进行训练与验证
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
#实例化模型
model = BertClassificationModel()
#调用训练函数进行训练与验证
train(model,train_loader,dev_loader)#需要全部在同一个device 
path = 'data/span_bert_hide_model1.pkl'
torch.save(model, path)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
Some weights of the model checkpoint at bert-base-chinese/ were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoi

开始训练!!!


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0:   0%|          | 0/449 [00:00<?, ?it/s]

Train Epoch[1/2],step[50/449],tra_acc72.321429 %,loss:0.555447
Train Epoch[1/2],step[100/449],tra_acc73.547980 %,loss:0.548377


...

DEV Epoch[2/2],step[200/449],tra_acc81.029653 %,bestAcc81.361607%,dev_acc81.361607 %,loss:0.146291
Train Epoch[2/2],step[250/449],tra_acc81.493768 %,loss:0.346931
Train Epoch[2/2],step[300/449],tra_acc81.920857 %,loss:0.362602



Dev Itreation:: 0it [00:00, ?it/s]
Dev Itreation:: 1it [00:00,  7.90it/s]
Dev Itreation:: 2it [00:00,  7.59it/s]
Dev Itreation:: 3it [00:00,  7.56it/s]
Dev Itreation:: 4it [00:00,  7.36it/s]
Dev Itreation:: 5it [00:00,  7.28it/s]
Dev Itreation:: 6it [00:00,  7.30it/s]
Dev Itreation:: 7it [00:00,  7.19it/s]
Dev Itreation:: 8it [00:01,  7.27it/s]
Dev Itreation:: 9it [00:01,  7.32it/s]
Dev Itreation:: 10it [00:01,  7.31it/s]
Dev Itreation:: 11it [00:01,  7.33it/s]
Dev Itreation:: 12it [00:01,  7.38it/s]
Dev Itreation:: 13it [00:01,  7.33it/s]
Dev Itreation:: 14it [00:01,  7.23it/s]
Dev Itreation:: 15it [00:02,  7.18it/s]
Dev Itreation:: 16it [00:02,  7.22it/s]
Dev Itreation:: 17it [00:02,  7.28it/s]
Dev Itreation:: 18it [00:02,  7.28it/s]
Dev Itreation:: 19it [00:02,  7.28it/s]
Dev Itreation:: 20it [00:02,  7.22it/s]
Dev Itreation:: 21it [00:02,  7.17it/s]
Dev Itreation:: 22it [00:03,  7.18it/s]
Dev Itreation:: 23it [00:03,  7.22it/s]
Dev Itreation:: 24it [00:03,  7.31it/s]
Dev Itreation

DEV Epoch[2/2],step[300/449],tra_acc81.920857 %,bestAcc81.473214%,dev_acc81.473214 %,loss:0.362602
Train Epoch[2/2],step[350/449],tra_acc82.317886 %,loss:0.359175
Train Epoch[2/2],step[400/449],tra_acc82.586881 %,loss:0.414539



Dev Itreation:: 0it [00:00, ?it/s]
Dev Itreation:: 1it [00:00,  7.76it/s]
Dev Itreation:: 2it [00:00,  7.45it/s]
Dev Itreation:: 3it [00:00,  7.38it/s]
Dev Itreation:: 4it [00:00,  7.34it/s]
Dev Itreation:: 5it [00:00,  7.39it/s]
Dev Itreation:: 6it [00:00,  7.30it/s]
Dev Itreation:: 7it [00:00,  7.24it/s]
Dev Itreation:: 8it [00:01,  7.31it/s]
Dev Itreation:: 9it [00:01,  7.39it/s]
Dev Itreation:: 10it [00:01,  7.36it/s]
Dev Itreation:: 11it [00:01,  7.36it/s]
Dev Itreation:: 12it [00:01,  7.40it/s]
Dev Itreation:: 13it [00:01,  7.34it/s]
Dev Itreation:: 14it [00:01,  7.28it/s]
Dev Itreation:: 15it [00:02,  7.31it/s]
Dev Itreation:: 16it [00:02,  7.34it/s]
Dev Itreation:: 17it [00:02,  7.30it/s]
Dev Itreation:: 18it [00:02,  7.25it/s]
Dev Itreation:: 19it [00:02,  7.23it/s]
Dev Itreation:: 20it [00:02,  7.16it/s]
Dev Itreation:: 21it [00:02,  7.22it/s]
Dev Itreation:: 22it [00:03,  7.22it/s]
Dev Itreation:: 23it [00:03,  7.28it/s]
Dev Itreation:: 24it [00:03,  7.25it/s]
Dev Itreation

DEV Epoch[2/2],step[400/449],tra_acc82.586881 %,bestAcc81.473214%,dev_acc81.473214 %,loss:0.414539
Train Epoch[2/2],step[450/449],tra_acc82.975310 %,loss:0.044038


In [42]:
#测试过程
path = 'data/span_bert_hide_model1.pkl'
Trained_model = torch.load(path)
acc=0
for batch in tqdm(test_loader,desc=f"Testing"):
    input_ids,token_type_ids,attention_mask,labels=[x.to(device) for x in batch]
    with torch.no_grad():
        output=Trained_model(input_ids,token_type_ids,attention_mask)
        acc+=(output.argmax(dim=1)==labels).sum().item()
print(f"\nAcc:{acc/len(test_data):.2f}")
#%%调用函数计算准确率等指标
#P = sklearn.metrics.precision_score(y_true, y_pred, average='binary',sample_weight=None)
#R = sklearn.metrics.recall_score(y_true, y_pred, average='binary',sample_weight=None)
#F1 = sklearn.metrics.f1_score(y_true, y_pred,average='binary',sample_weight=None)       

Testing: 100%|██████████| 57/57 [00:07<00:00,  7.19it/s]


Acc:0.80


In [43]:
print(f"\nAcc:{acc/len(test_data):.2f}")


Acc:0.80


In [44]:
len(test_data)

897